In [16]:
import pandas as pd

In [17]:
from XrayTo3DShape import VerseExcelSheet, VerseKeys

In [14]:
verse_metadata = VerseExcelSheet('/mnt/SSD0/mahesh-home/xrayto3D-benchmark/meta_results/ryai190138_appendixe1.xlsx')

In [15]:
def get_shape(row):
    """shape lambda"""
    return verse_metadata.get_shape(verse_metadata.get_vertebra_keys(row["subject-id"]))


def get_severity(row):
    """severity lambda"""
    return verse_metadata.get_severity(
        verse_metadata.get_vertebra_keys(row["subject-id"])
    )


def get_vertebra_level(row):
    """vertebra level"""
    return verse_metadata.get_vertebra_level(
        verse_metadata.get_vertebra_keys(row["subject-id"])
    )


In [9]:
EVAL_LOG_CSV_PATH_TEMPLATE = "/mnt/SSD0/mahesh-home/xrayto3D-benchmark/runs/2d-3d-benchmark/{run_id}/evaluation/metric-log.csv"

In [25]:
comparison_runs = {'rand-init':('p3qkfyj5',"/mnt/SSD0/mahesh-home/xrayto3D-benchmark/runs/2d-3d-benchmark/{run_id}/evaluation/metric-log.csv"),
'transfer-learn':('czyht2ac',"/mnt/SSD0/mahesh-home/Towards-Foundation-Model-for-XrayTo3D/runs/2d-3d-foundation-model/{run_id}/evaluation/metric-log.csv")}

In [29]:
for model_type, (run_id,log_template) in comparison_runs.items():
    input_log = EVAL_LOG_CSV_PATH_TEMPLATE.format(run_id=run_id)
    df = pd.read_csv(input_log)
    df["shape"] = df.apply(get_shape, axis="columns")
    df["severity"] = df.apply(get_severity, axis="columns")
    df["level"] = df.apply(get_vertebra_level, axis="columns")
    row = f"{model_type:20s}"
    for metric_type in ["DSC", "HD95", "NSD"]:
        for level_type in [
            VerseKeys.CERVICAL,
            VerseKeys.THORACIC,
            VerseKeys.LUMBAR,
        ]:
            row += f'& {df.groupby(["level"])[metric_type].mean()[level_type]:5.2f}'
    row += r"\\"  # latex new line
    print('level\n',row)

    row = f"{model_type:20s}"
    for metric_type in ["DSC", "HD95", "NSD"]:
        for level_type in [
            VerseKeys.NORMAL,
            VerseKeys.WEDGE,
            VerseKeys.BICONCAVE,
            VerseKeys.CRUSH,
        ]:
            row += f'& {df.groupby(["shape"])[metric_type].mean()[level_type]:5.2f}'
    row += r"\\"  # latex new line    
    print('shape\n',row)
    row = f"{model_type:20s}"
    for metric_type in ["DSC", "HD95", "NSD"]:
        for level_type in [
            VerseKeys.NORMAL,
            VerseKeys.MILD,
            VerseKeys.MODERATE,
            VerseKeys.SEVERE,
        ]:
            row += f'& {df.groupby(["severity"])[metric_type].mean()[level_type]:5.2f}'
    row += r"\\"  # latex new line
    print('severity\n',row)

level
 rand-init           &  0.75&  0.84&  0.86&  3.07&  2.37&  2.29&  0.81&  0.87&  0.85\\
shape
 rand-init           &  0.84&  0.86&  0.84&  0.78&  2.38&  2.23&  2.67&  3.13&  0.86&  0.87&  0.83&  0.74\\
severity
 rand-init           &  0.84&  0.88&  0.84&  0.74&  2.38&  1.77&  2.48&  4.37&  0.86&  0.91&  0.85&  0.65\\
level
 transfer-learn      &  0.77&  0.86&  0.88&  2.75&  2.01&  2.17&  0.83&  0.89&  0.87\\
shape
 transfer-learn      &  0.85&  0.88&  0.87&  0.83&  2.13&  1.88&  2.16&  2.56&  0.88&  0.91&  0.87&  0.82\\
severity
 transfer-learn      &  0.85&  0.89&  0.87&  0.81&  2.13&  1.74&  2.02&  2.99&  0.88&  0.92&  0.89&  0.77\\
